# L-BFGS

L-BFGS is a classical optimization method that uses past gradients and parameters informations to iteratively refine a solution to a minimization problem. In this notebook, we illustrate
1. how to use L-BFGS as a simple gradient transformation,
2. how to wrap L-BFGS in a solver, and how linesearches are incorporated,
3. how to debug the solver if needed,
3. how to use L-BFGS to train a medium scale network on CIFAR10.

In [ ]:
from typing import NamedTuple

import chex
import jax
import jax.numpy as jnp
import jax.random as jrd

import optax
import optax.tree_utils as otu

## L-BFGS as a gradient transformation


### What is L-BFGS?

To solve a problem of the form
$$
\min_w f(w),
$$
L-BFGS ([Limited memory Broyden–Fletcher–Goldfarb–Shanno algorithm](https://en.wikipedia.org/wiki/Limited-memory_BFGS)) makes steps of the form
$$
w_{k+1} = w_k - \eta_k P_k g_k,
$$
where, at iteration $k$, $w_k$ are the parameters, $g_k = \nabla f_k$ are the gradients, $\eta_k$ is the stepsize, and $P_k$ is a *preconditioning* matrix, that is, a matrix that transforms the gradients to ease the optimization process.

L-BFGS builds the preconditioning matrix $P_k$ as an approximation of the Hessian inverse $P_k \approx \nabla^2 f(w_k)^{-1}$ using past gradient and parameters information. Briefly, at iteration $k$, the previous preconditioning matrix $P_{k-1}$ is updated such that $P_k$ satisfies the secant condition $P_k(w_k-w_{k-1}) = g_k -g_{k-1}$. The original BFGS algorithm updates $P_k$ using all past information, the limited-memory variant only uses a fixed number of past parameters and gradients to build $P_k$. See [Nocedal and Wright, Numerical Optimization, 1999](https://www.math.uci.edu/~qnie/Publications/NumericalOptimization.pdf) or the [documentation](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_lbfgs) for more details on the implementation.


### Using L-BFGS as a gradient transformation

The function [optax.scale_by_lbfgs](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_lbfgs) implements the update of the preconditioning matrix given a running optimizer state $s_k$. Given $(g_k, s_k, w_k)$, this function returns $(P_kg_k, s_{k+1})$. We illustrate its performance below on a simple convex quadratic.

In [ ]:
# Define objective
dim = 8
w_opt = jnp.ones(dim)
mat = jrd.normal(jrd.PRNGKey(0), (dim, dim))
mat = mat.dot(mat.T)


def fun(w):
  return 0.5 * (w - w_opt).dot(mat.dot(w - w_opt))


# Define optimizer
lr = 1e-1
opt = optax.scale_by_lbfgs()

# Initialize optimization
w = jrd.normal(jrd.PRNGKey(1), (dim,))
state = opt.init(w_opt)

# Run optimization
for i in range(16):
  v, g = jax.value_and_grad(fun)(w)
  print(f'Iteration: {i}, Value:{v:.2e}')
  u, state = opt.update(g, state, w)
  w = w - lr * u

print(f'Final value: {fun(w):.2e}')

## L-BFGS as a solver

L-BFGS is a stample in numerical optimization to solve medium scale problems. It is often the backend of generic minimization functions in software libraries like [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize). A key ingredient to make it a simple optimization blackbox, is to remove the need of tuning the stepsize, a.k.a. learning rate in machine learning. In a deterministic setting (no additional varying inputs like inputs/labels), such automatic tuning of the stepsize is done by means of linesearches reviewed below.

### What are linesearches?

Given current parameters $w_k$, an update direction $u_k$ (such as the negative preconditioned gradient $u_k = -P_k g_k$ returned by L-BFGS), a linesearch computes a stepsize $\eta_k$ such that the next iterate $w_{k+1} = w_k + \eta_k u_k$ satisfies some criterions.

#### Sufficient decrease (Armijo-Goldstein criterion)

The first criterion that a good stepsize may need to satisfy is to ensure that the next iterate decreases the value of the objective by a a sufficient amount. Mathematically, the criterion is expressed as finding $\eta_k$ such that

$$
f(w_k + \eta_k u_k) \leq f(w_k) + c_1 \eta_k \langle u_k, g_k\rangle
$$

where $c_1$ is some constant set to $10^{-4}$ by default. Consider for example the update direction to be $u_k = -g_k$, i.e., moving along the negative gradient direction. In that case the criterion above reduces to $f(w_k - \eta_k g_k) \leq f(w_k) - c_1 \eta_k ||g_k||_2^2$. The criterion amounts then to choosing the stepsize such that it decreases the objective by an amount proportional to the squared gradient norm.

As long as the update direction is a *descent direction*, that is, $\langle u_k, g_k\rangle < 0$ the above criterion is guaranteed to be satisfied by some sufficiently small stepsize.
A simple linesearch technique to ensure a sufficient decrease is then to decrease a candidate stepsize by a constant factor up until the criterion is satisfied. This amounts to the backtracking linesearch implemented in [optax.scale_by_backtracking_linesearch](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_backtracking_linesearch) and briefly reviewed below.

#### Small curvature (Strong wolfe criterion)

The sufficient decrease criterion ensures that the algorithm does not produce a sequence of diverging objective values. However, we may want to not only reduce a current stepsize but also increase it to ensure maximal speed. Ideally, we would like to find the stepsize that minimizes the function along the current update, i.e., $\eta_k^* = \arg\min_\eta f(w_k + \eta u_k)$. Such an endeavor can be computationally prohibitive, so we may select a stepsize that ensures some properties that an optimal stepsize would satisfy. In particular, we may search for a stepsize such that the derivative of $h(\eta) = f(w_k + \eta u_k)$ is small enough compared to its derivativeœ at $\eta=0$. Formally, we may want to select the stepsize $\eta_k$ such that $|h'(eta_k)| \leq |h'(0)|$, that is,

$$
|\langle \nabla f(w_k + \eta_k u_k), u_k\rangle|
\leq |\langle \nabla f(w_k), u_k\rangle|.
$$

See Chapter 3 of [Nocedal and Wright, Numerical Optimization, 1999](https://www.math.uci.edu/~qnie/Publications/NumericalOptimization.pdf) for some illustrations of this criterion. A linesearch method that can ensure both criterions require some form of bisection method implemented in optax with the [optax.scale_by_zoom_linesearch](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_zoom_linesearch) method. Several other linesearch techniques exist, see e.g. https://github.com/JuliaNLSolvers/LineSearches.jl. It is generally recommended to combine L-BFGS with a line-search ensuring both sufficient decrease and small curvature, which the [optax.scale_by_zoom_linesearch](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_zoom_linesearch) ensures.



### Linesearches in practice

To find a stepsize satisfying the above criterions, a linesearch needs to access the value and potentially the gradient of the function. So linesearches in optax are implemented as [optax.GradientTransformationExtraArgs](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.GradientTransformationExtraArgs), which take the current value, gradient of the objective as well as the function itself. We illustrate this below with [optax.scale_by_backtracking_linesearch](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_backtracking_linesearch).

In [ ]:
# Objective
def fun(w):
  return jnp.sum(jnp.abs(w))


# Linesearch, comment/uncomment the desired one
linesearch = optax.scale_by_backtracking_linesearch(max_backtracking_steps=15)
# linesearch = optax.scale_by_zoom_linesearch(max_linesearch_steps=15)

# Optimizer
opt = optax.chain(
    optax.sgd(learning_rate=1.0),
    # Compare with or without linesearch by commenting this line
    linesearch,
)

# Initialize
w = jrd.normal(jrd.PRNGKey(0), (8,))
state = opt.init(w)

# Run optimization
for i in range(16):
  v, g = jax.value_and_grad(fun)(w)
  print(f'Iteration: {i}, Value:{v:.2e}')
  u, state = opt.update(g, state, w, value=v, grad=g, value_fn=fun)
  w = w + u

print(f'Final value: {fun(w):.2e}')

To validate the stepsize the linesearch calls the function several times. If a stepsize is accepted, we have then a priori access to the value of the function, and, potentially its gradient. The implementation of the linesearches in optax store the value and the gradient computed by the linesearch to avoid recomputing them at the next step. In practice, the code above can be modified as follows.

*Note:*
The backtracking linesearch only evaluates the function and does not compute the gradient natively. To make the backtracking linesearch compute and store the gradient at the stepsize taken, we add the flag `store_grad=True`, see below.
The zoom linesearch always compute both function and gradient so there is no need to specify an additional flag.

In [ ]:
# Objective
def fun(w):
  return jnp.sum(jnp.abs(w))


# Linesearch
linesearch = optax.scale_by_backtracking_linesearch(
    max_backtracking_steps=15, store_grad=True
)
# linesearch = optax.scale_by_zoom_linesearch(max_linesearch_steps=15)

# Optimizer
opt = optax.chain(optax.sgd(learning_rate=1.0), linesearch)

# Initialize
w = jrd.normal(jrd.PRNGKey(0), (8,))
state = opt.init(w)

# Run optimization
for _ in range(16):
  # Replace `v, g = jax.value_and_grad(fun)(w)` by
  v, g = optax.value_and_grad_from_state(fun)(w, state=state)
  u, state = opt.update(g, state, w, value=v, grad=g, value_fn=fun)
  w = w + u

print(f'Final value: {fun(w):.2e}')

### L-BFGS solver

Optax combines then the gradient transformation of L-BFGS and a linesearch in `optax.lbfgs()`.

We present below a wrapper that combines both into a solver which tries to find the minimizer of a function given
1. some initial parameters `init_params`,
2. the function to optimize `fun`,
3. the instance of the L-BFGS solver considered `opt`,
4. a maximal number of iteration `max_iter`,
5. a tolerance `tol` on the optimization error measured here as the gradient norm.

In [ ]:
def run_lbfgs(init_params, fun, opt, max_iter, tol):
  value_and_grad_fun = optax.value_and_grad_from_state(fun)

  def step(carry):
    params, state = carry
    value, grad = value_and_grad_fun(params, state=state)
    updates, state = opt.update(
        grad, state, params, value=value, grad=grad, value_fn=fun
    )
    params = optax.apply_updates(params, updates)
    return params, state

  def continuing_criterion(carry):
    _, state = carry
    iter_num = otu.tree_get(state, 'count')
    grad = otu.tree_get(state, 'grad')
    err = otu.tree_l2_norm(grad)
    return (iter_num == 0) | ((iter_num < max_iter) & (err >= tol))

  init_carry = (init_params, opt.init(init_params))
  final_params, final_state = jax.lax.while_loop(
      continuing_criterion, step, init_carry
  )
  return final_params, final_state

We can test the solver on the [Rosenbrock function](https://www.sfu.ca/~ssurjano/rosen.html).

In [ ]:
def fun(w):
  return jnp.sum(100.0 * (w[1:] - w[:-1] ** 2) ** 2 + (1.0 - w[:-1]) ** 2)

opt = optax.lbfgs()
init_params = jnp.zeros((8,))
print(
    f'Initial value: {fun(init_params):.2e} '
    f'Initial gradient norm: {otu.tree_l2_norm(jax.grad(fun)(init_params)):.2e}'
)
final_params, _ = run_lbfgs(init_params, fun, opt, max_iter=50, tol=1e-3)
print(
    f'Final value: {fun(final_params):.2e}, '
    f'Final gradient norm: {otu.tree_l2_norm(jax.grad(fun)(final_params)):.2e}'
)

We may add additional information by simply chaining `optax.lbfgs` with an identity transform that just prints relevant information as follows.

In [ ]:
class InfoState(NamedTuple):
  iter_num: chex.Numeric


def print_info():
  def init_fn(params):
    del params
    return InfoState(iter_num=0)

  def update_fn(updates, state, params, *, value, grad, **extra_args):
    del params, extra_args

    jax.debug.print(
        'Iteration: {i}, Value: {v}, Gradient norm: {e}',
        i=state.iter_num,
        v=value,
        e=otu.tree_l2_norm(grad),
    )
    return updates, InfoState(iter_num=state.iter_num + 1)

  return optax.GradientTransformationExtraArgs(init_fn, update_fn)


def fun(w):
  return jnp.sum(100.0 * (w[1:] - w[:-1] ** 2) ** 2 + (1.0 - w[:-1]) ** 2)


opt = optax.chain(print_info(), optax.lbfgs())
init_params = jnp.zeros((8,))
print(
    f'Initial value: {fun(init_params):.2e} '
    f'Initial gradient norm: {otu.tree_l2_norm(jax.grad(fun)(init_params)):.2e}'
)
final_params, _ = run_lbfgs(init_params, fun, opt, max_iter=50, tol=1e-3)
print(
    f'Final value: {fun(final_params):.2e}, '
    f'Final gradient norm: {otu.tree_l2_norm(jax.grad(fun)(final_params)):.2e}'
)

## Debugging solver

In some cases, L-BFGS with a linesearch as a solver will fail. Most of the times, the culprit goes down to the linesearch. To debug the solver in such cases, we provide a `verbose` option to the `optax.scale_by_zoom_linesearch`. We show below how to proceed.

First we try to minimize the [Zakharov function](https://www.sfu.ca/~ssurjano/zakharov.html) without any changes. You'll observe that the final value is larger than the initial value which points out that the solver failed, and probably because the linesearch did not find a stepsize that ensured a sufficient decrease.

In [ ]:
def fun(w):
  ii = jnp.arange(1, len(w) + 1, step=1, dtype=w.dtype)
  sum1 = (w**2).sum()
  sum2 = (0.5 * ii * w).sum()
  return sum1 + sum2**2 + sum2**4

opt = optax.chain(print_info(), optax.lbfgs())

init_params = jnp.array([600.0, 700.0, 200.0, 100.0, 90.0, 1e4])
print(
    f'Initial value: {fun(init_params)} '
    f'Initial gradient norm: {otu.tree_l2_norm(jax.grad(fun)(init_params))}'
)
final_params, _ = run_lbfgs(init_params, fun, opt, max_iter=50, tol=1e-3)
print(
    f'Final value: {fun(final_params)}, '
    f'Final gradient norm: {otu.tree_l2_norm(jax.grad(fun)(final_params))}'
)

We can change the linesearch used in lbfgs as part of its arguments. Here we keep the default number of linesearch steps (15) and set the verbose option to `True`.

In [ ]:
opt = optax.chain(print_info(), optax.lbfgs(
  linesearch=optax.scale_by_zoom_linesearch(
      max_linesearch_steps=15, verbose=True
  )
))

init_params = jnp.array([600.0, 700.0, 200.0, 100.0, 90.0, 1e4])
print(
    f'Initial value: {fun(init_params):.2e} '
    f'Initial gradient norm: {otu.tree_l2_norm(jax.grad(fun)(init_params)):.2e}'
)
final_params, _ = run_lbfgs(init_params, fun, opt, max_iter=50, tol=1e-3)
print(
    f'Final value: {fun(final_params):.2e}, '
    f'Final gradient norm: {otu.tree_l2_norm(jax.grad(fun)(final_params)):.2e}'
)

As expected, the linesearch failed at the very first step taking a stepsize that did not ensure a sufficient decrease. Multiple information is displayed. For example, the slope (derivative along the update direction) at the first step si extremely large which explains the difficulties to find an appropriate stepsize. As pointed out in the log above, the first thing to try is to use a larger number of linesearch steps.

In [ ]:
opt = optax.chain(print_info(), optax.lbfgs(
  linesearch=optax.scale_by_zoom_linesearch(
      max_linesearch_steps=50, verbose=True
  )
))

init_params = jnp.array([600.0, 700.0, 200.0, 100.0, 90.0, 1e4])
print(
    f'Initial value: {fun(init_params):.2e} '
    f'Initial gradient norm: {otu.tree_l2_norm(jax.grad(fun)(init_params)):.2e}'
)
final_params, _ = run_lbfgs(init_params, fun, opt, max_iter=50, tol=1e-3)
print(
    f'Final value: {fun(final_params):.2e}, '
    f'Final gradient norm: {otu.tree_l2_norm(jax.grad(fun)(final_params)):.2e}'
)

By simply taking a maximum of 50 steps of the linesearch instead of 15, we ensured that the first stepsize taken provided a sufficient decrease and the solver worked well.
Additional debugging information can be found in the source code accessible from the docs of [optax.scale_by_zoom_linesearch](https://optax.readthedocs.io/en/latest/api/transformations.html#optax.scale_by_zoom_linesearch).